In [1]:
import numpy as np
from parse_lsp_data import *
import tensorflow as tf

/home/shared/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from keras.preprocessing import image
from keras import applications
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.models import Model,Input
from keras.utils import np_utils
from keras.models import Sequential
from keras.callbacks import CSVLogger
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout,Reshape
from keras.optimizers import SGD,Adam
import keras.backend as K
from keras.layers import Input, Dense, Dropout, Conv2D, MaxPooling2D, Flatten,Activation, Average,GlobalAveragePooling2D

Using TensorFlow backend.


In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2150996135503141571
]


In [4]:
# get the data set
train_list, train_label, val_list, val_label, test_list, test_label = getLSPDataset(0.8, 0.1)

Resizing and packing images and labels to lists.

Start to process training dataset
Done processing the training dataset
Start to process validation dataset
Done processing the validation dataset
Start to process test dataset
Done processing the test dataset


In [5]:
# print some information
train_X = train_list	# (N, 227, 227, 3)
train_Y = train_label	# (N, 2, 14)
print("train X: ", train_list.shape, "val X: ", val_list.shape, "test X: ", test_list.shape)
print("train Y: ", train_label.shape, "val Y: ", val_label.shape, "test Y: ", test_label.shape)

train X:  (1600, 227, 227, 3) val X:  (200, 227, 227, 3) test X:  (200, 227, 227, 3)
train Y:  (1600, 2, 14) val Y:  (200, 2, 14) test Y:  (200, 2, 14)


In [6]:
# model_input = Input(shape=(227,227,3))

In [7]:
# build keras model
def build_model():
#     # Block 1
#     x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(model_input)
#     x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
#     x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

#     # Block 2
#     x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
#     x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
#     x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

#     # Block 3
#     x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
#     x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
#     x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
#     x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

#     # Block 4
#     x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
#     x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
#     x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
#     x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

#     # Block 5
#     x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
#     x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
#     x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
#     x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)
    
#     x = Flatten(name='flatten')(x)
#     x = Dense(4096, activation='relu', name='fc1')(x)
#     x = Dense(4096, activation='relu', name='fc2')(x)
#     x = Dense(28, activation='relu', name='fc3')(x)
#     x = Reshape((2,14))(x)
#     model = Model(model_input, x, name='vgg16')
    base_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(227,227, 3))
    last = base_model.output
    x = Flatten()(last)
    x = Dense(4096, activation='relu')(x)
#     x = Dropout(0.5)(x)
    x = Dense(4096, activation='relu')(x)
    x = Dense(28, activation='relu')(x)
    preds = Reshape((2,14))(x)
    model = Model(base_model.input, preds)
    return model

In [8]:
model = build_model()

94658560/94653016 [==============================] - 9s 0us/step


In [9]:
#model.summary()

In [10]:
# custom loss function (L2 distance)
def euclidean_distance_loss(y_true, y_pred):
    return K.sqrt(K.sum(K.square(y_pred - y_true), axis=-1))

In [11]:
def loss_accuracy(y_true, y_pred):
    diff = tf.square(y_true - y_pred)
    loss = tf.reduce_sum(diff)
    dist = tf.sqrt(tf.reduce_sum(diff, axis = 1))	# (N, 2, 14) -> (N, 14)
    accuracy = tf.reduce_sum(tf.to_int32(tf.greater(1.0, dist))) / 128/14
    return accuracy

In [12]:
# compile the model
model.compile(optimizer=Adam(), loss=euclidean_distance_loss, metrics=[loss_accuracy])

In [13]:
csv_logger = CSVLogger('logvgg_temp.csv', append=True, separator=';')

In [14]:
with tf.device('/device:GPU:0'):
    # fit the model
    history = model.fit(train_X, train_Y, epochs=100,batch_size=64, validation_data = (val_list,val_label),callbacks=[csv_logger])

Train on 1600 samples, validate on 200 samples
Epoch 1/100
1600/1600 [==============================] - 36s 23ms/step - loss: 170.3901 - loss_accuracy: 2.2321e-04 - val_loss: 216.3692 - val_loss_accuracy: 0.0000e+00
Epoch 2/100
1600/1600 [==============================] - 13s 8ms/step - loss: 113.6698 - loss_accuracy: 2.0089e-04 - val_loss: 191.7068 - val_loss_accuracy: 0.0000e+00
Epoch 3/100
1600/1600 [==============================] - 13s 8ms/step - loss: 104.7751 - loss_accuracy: 6.9196e-04 - val_loss: 129.7079 - val_loss_accuracy: 1.7857e-04
Epoch 4/100
1600/1600 [==============================] - 13s 8ms/step - loss: 101.6837 - loss_accuracy: 6.4732e-04 - val_loss: 105.0358 - val_loss_accuracy: 3.5714e-04
Epoch 5/100
1600/1600 [==============================] - 13s 8ms/step - loss: 90.1829 - loss_accuracy: 9.1518e-04 - val_loss: 118.7286 - val_loss_accuracy: 0.0013
Epoch 6/100
1600/1600 [==============================] - 13s 8ms/step - loss: 83.6011 - loss_accuracy: 9.3750e-04 - v

In [17]:
# evaluation
with tf.device('/device:GPU:0'):
    loss,evaluation_metric_loss = model.evaluate(test_list, test_label)

200/200 [==============================] - 1s 4ms/step


In [18]:
print(loss)

57.74504180908203


In [15]:
with tf.device('/device:GPU:0'):
    pred = model.predict(test_list)

In [16]:
print(pred[0])
print(train_Y[0])

[[ 46.973225  84.177     83.005135 125.44217  169.66284  106.70624
  129.22632   55.823048  92.20912  169.3768   140.30469  174.49522
  140.66011  152.4027  ]
 [208.1998   161.95995  113.92505  112.64477  163.4359   196.07788
   84.65019   82.85775   50.968502  53.577644  75.67746   90.58651
   44.530647  17.0624  ]]
[[ 50.20740877  83.82652876  85.02721161 130.65316017 172.67706016
  105.43882018 130.65316017  61.01355448  95.83335733 170.27569445
  142.65998874 173.87774302 141.45930588 151.06476874]
 [206.30842279 161.39537968 115.06402995 115.53679882 160.92261081
  191.65258767  83.38851533  84.80682196  53.60407622  53.60407622
   74.87867558  87.64343521  47.45808084  19.56471723]]


In [ ]:
dist = K.sqrt(K.sum(K.square(pred[0] - test_label[0]), axis=0))

In [ ]:
threshold = np.array(1.0)

In [ ]:
temp = tf.reduce_sum(tf.to_int32(tf.greater(threshold, dist)))

In [ ]:
print(temp.shape)